In [1]:
from github import Github
import pandas as pd

In [2]:
%run settings.ipynb

token = %env token

github = Github(token)

env: token=975a853da92841049fd9ceba4b4e32ec9a32abec


## Análise de testes usados nos projetos do dataset.

1. Análise de quantos projetos usam arquivos de controle.
2. Armazenar arquivos de controle.
3. Verificar se existe algum pacote de teste no arquivo de controle.
4. Verificar qual pacote está sendo usado.


In [3]:
tools = pd.read_csv('dataset/de_para_tools.csv', sep=';')

tools['language'].unique()


array(['JavaScript', 'Python', 'Java', 'Go', 'Ruby', 'PHP', 'TypeScript',
       'C#'], dtype=object)

In [5]:
manager_file = {}

for t in tools['language'].unique():
    manager_file[t] = {"yes": 0, "total": 0}

manager_file

{'C#': {'total': 0, 'yes': 0},
 'Go': {'total': 0, 'yes': 0},
 'Java': {'total': 0, 'yes': 0},
 'JavaScript': {'total': 0, 'yes': 0},
 'PHP': {'total': 0, 'yes': 0},
 'Python': {'total': 0, 'yes': 0},
 'Ruby': {'total': 0, 'yes': 0},
 'TypeScript': {'total': 0, 'yes': 0}}

In [20]:
manager_package = {}

for t in tools['language'].unique():
    manager_package[t] = {}
    for p in tools[tools['language'] == t]['test_package']:
        manager_package[t][p] = 0

manager_package

{'C#': {'CoreClr': 0, 'NUnit': 0, 'NUnit3TestAdapter': 0},
 'Go': {'Testify': 0,
  'assert': 0,
  'biff': 0,
  'check': 0,
  'convey': 0,
  'e2e': 0,
  'ginkgo': 0,
  'go-check': 0,
  'go-testing-interface': 0,
  'gocheck': 0,
  'gomega': 0,
  'gucumber': 0,
  'require': 0,
  'spew': 0,
  'suite': 0,
  'testing': 0,
  'testutil': 0},
 'Java': {'EasyMock': 0,
  'hamcrest-all': 0,
  'junit': 0,
  'mockito-all': 0,
  'mockito-core': 0,
  'scalatest': 0,
  'spring-test': 0,
  'testNG': 0},
 'JavaScript': {'Appium': 0,
  'Cypress': 0,
  'Electron': 0,
  'Gemini': 0,
  'Happo': 0,
  'Istanbul': 0,
  'Nightwatch': 0,
  'Percy': 0,
  'PhantomJS': 0,
  'Protractor': 0,
  'Puppeteer': 0,
  'Sinon.JS': 0,
  'TestCafe': 0,
  'Unexpected': 0,
  'ava': 0,
  'chai': 0,
  'cucumber': 0,
  'cypress': 0,
  'enzyme': 0,
  'enzyme-adapter-react-16': 0,
  'enzyme-adapter-utils': 0,
  'jest': 0,
  'jest-cli': 0,
  'jsdom': 0,
  'json-server': 0,
  'karma': 0,
  'mocha': 0,
  'nyc': 0,
  'protractor': 0,
  '

### TO-DO: Atualização Dataset.

Filtro para restringir dataset para apenas a lista de linguagens definidas.

In [15]:
repo_full = pd.read_csv('dataset/repositories.csv', sep=';')
#repo_full.drop("Unnamed: 0", axis=1, inplace=True) 
repo_full.head()

,full_name,git_url,id,language
0,b4winckler/macvim,git://github.com/b4winckler/macvim.git,692798,C
1,php/php-src,git://github.com/php/php-src.git,1903522,C
2,antirez/redis,git://github.com/antirez/redis.git,156018,C
3,libgit2/libgit2,git://github.com/libgit2/libgit2.git,901662,C
4,torvalds/linux,git://github.com/torvalds/linux.git,2325298,C


### Quantidade total de projetos de uma mesma linguagem.

In [7]:
for index, row in repo_full.iterrows():
    if row['language'] in manager_file:
        manager_file[row['language']]["total"]+=1

manager_file

{'C#': {'total': 13, 'yes': 0},
 'Go': {'total': 33, 'yes': 0},
 'Java': {'total': 38, 'yes': 0},
 'JavaScript': {'total': 81, 'yes': 0},
 'PHP': {'total': 21, 'yes': 0},
 'Python': {'total': 52, 'yes': 0},
 'Ruby': {'total': 23, 'yes': 0},
 'TypeScript': {'total': 15, 'yes': 0}}

#### Arquivo de controle de pacotes de teste.

In [12]:
tools[tools['language'] == 'Java']

,language,test_package,file_packages_installed
43,Java,junit,pom.xml
44,Java,testNG,pom.xml
45,Java,EasyMock,pom.xml
46,Java,mockito-all,pom.xml
47,Java,mockito-core,pom.xml
48,Java,scalatest,pom.xml
49,Java,spring-test,pom.xml
50,Java,hamcrest-all,pom.xml


### Quantidade total de projetos que possui o arquivo de gerencia de pacotes.

In [22]:
import requests

for index, row in repo_full.iterrows():
    packages = tools[tools['language'] == row['language']]
    arq = packages['file_packages_installed'].unique()

    if(len(arq) > 0):
        #Pega o repositório pelo nome.
        repo = github.get_repo(row['full_name'])
        print(row['full_name'])
        try:
            print(row['language'])
            #Pegar um arquivo específico.
            content = repo.get_contents(str(arq[0]))
            print(content.path)
            #Registrar quantidade de projetos possui arquivo de controle.
            manager_file[row['language']]["yes"]+=1
            
            #Obter conteúdo do arquivo fazendo requisicao direta a API.
            #print('https://raw.githubusercontent.com/' + row['full_name'] + '/master/' +  content.path)
            response = requests.get('https://raw.githubusercontent.com/' + row['full_name'] + '/master/' +  content.path)
            response.encoding = 'utf-8'
            texto = response.text
            
            #Verificar se possui um dos pacotes da lista de controle.
            for p in packages['test_package']:
                manager_package[row['language']][p]+=texto.count(p)
            
        except:

            pass


manager_file
manager_package

dotnet/corefx
C#
NuGet.config
OpenRA/OpenRA
C#
dotnet/roslyn
C#
NuGet.config
PowerShell/PowerShell
C#
NancyFx/Nancy
C#
NuGet.config
dotnet/coreclr
C#
NuGet.config
aspnet/EntityFrameworkCore
C#
NuGet.config
MonoGame/MonoGame
C#
mono/mono
C#
NuGet.config
aspnetboilerplate/aspnetboilerplate
C#
aspnet/AspNetCore
C#
NuGet.config
AvaloniaUI/Avalonia
C#
ShareX/ShareX
C#
kubernetes/kubernetes
Go
go.sum
golang/go
Go
cockroachdb/cockroach
Go
moby/moby
Go
influxdata/influxdb
Go
go.sum
openshift/origin
Go
hashicorp/terraform
Go
go.sum
vitessio/vitess
Go
etcd-io/etcd
Go
go.sum
hashicorp/packer
Go
go.sum
mattermost/mattermost-server
Go
go.sum
ethereum/go-ethereum
Go
ipfs/go-ipfs
Go
go.sum
hashicorp/vault
Go
go.sum
hashicorp/consul
Go
go.sum
hyperledger/fabric
Go
elastic/beats
Go
pingcap/tidb
Go
go.sum
kubernetes/kops
Go
flynn/flynn
Go
go-gitea/gitea
Go
go.sum
git-lfs/git-lfs
Go
go.sum
linuxkit/linuxkit
Go
istio/istio
Go
go.sum
weaveworks/weave
Go
getlantern/lantern
Go
goharbor/harbor
Go
prometheus/p

activeadmin/activeadmin
Ruby
github/linguist
Ruby
DefinitelyTyped/DefinitelyTyped
TypeScript
package.json
Microsoft/vscode
TypeScript
package.json
Microsoft/TypeScript
TypeScript
package.json
grafana/grafana
TypeScript
package.json
desktop/desktop
TypeScript
package.json
BabylonJS/Babylon.js
TypeScript
package.json
ant-design/ant-design
TypeScript
package.json
angular/angular
TypeScript
package.json
tinymce/tinymce
TypeScript
package.json
ionic-team/ionic
TypeScript
package.json
Foundry376/Mailspring
TypeScript
package.json
primefaces/primeng
TypeScript
package.json
apollographql/apollo-client
TypeScript
package.json
angular/material2
TypeScript
package.json
NativeScript/NativeScript
TypeScript
package.json


{'C#': {'total': 13, 'yes': 48},
 'Go': {'total': 33, 'yes': 34},
 'Java': {'total': 38, 'yes': 32},
 'JavaScript': {'total': 81, 'yes': 73},
 'PHP': {'total': 21, 'yes': 19},
 'Python': {'total': 52, 'yes': 16},
 'Ruby': {'total': 23, 'yes': 0},
 'TypeScript': {'total': 15, 'yes': 15}}

In [23]:
manager_package

{'C#': {'CoreClr': 0, 'NUnit': 0, 'NUnit3TestAdapter': 0},
 'Go': {'Testify': 0,
  'assert': 43,
  'biff': 0,
  'check': 75,
  'convey': 34,
  'e2e': 14,
  'ginkgo': 48,
  'go-check': 8,
  'go-testing-interface': 23,
  'gocheck': 2,
  'gomega': 41,
  'gucumber': 0,
  'require': 0,
  'spew': 47,
  'suite': 13,
  'testing': 33,
  'testutil': 3},
 'Java': {'EasyMock': 0,
  'hamcrest-all': 5,
  'junit': 91,
  'mockito-all': 1,
  'mockito-core': 10,
  'scalatest': 4,
  'spring-test': 2,
  'testNG': 0},
 'JavaScript': {'Appium': 0,
  'Cypress': 0,
  'Electron': 0,
  'Gemini': 1,
  'Happo': 0,
  'Istanbul': 0,
  'Nightwatch': 0,
  'Percy': 0,
  'PhantomJS': 0,
  'Protractor': 0,
  'Puppeteer': 0,
  'Sinon.JS': 0,
  'TestCafe': 0,
  'Unexpected': 0,
  'ava': 29,
  'chai': 49,
  'cucumber': 0,
  'cypress': 1,
  'enzyme': 28,
  'enzyme-adapter-react-16': 9,
  'enzyme-adapter-utils': 1,
  'jest': 220,
  'jest-cli': 20,
  'jsdom': 22,
  'json-server': 0,
  'karma': 205,
  'mocha': 122,
  'nyc': 20

In [50]:
import requests
response = requests.get('https://raw.githubusercontent.com/octokit/octokit.rb/master/Gemfile')
response.encoding = 'utf-8'
texto = response.text
print(texto)
print(texto.count("gemspec"))
#file_response = open("dataset/manager_file/teste.txt","w+")
#file_response.write(texto, "w+")
#file_response.close()


source 'https://rubygems.org'

gem 'jruby-openssl', :platforms => :jruby
gem 'rake', '> 11.0.1', '< 12.0'

group :development do
  gem 'awesome_print', :require => 'ap'
  gem 'guard-rspec', '~> 4.5'
  gem 'hirb-unicode'
  gem 'pry'
  gem 'redcarpet'
  gem 'wirb'
  gem 'wirble'
  gem 'yard'
end

group :test do
  gem 'coveralls', :require => false
  gem 'json', '~> 1.7', :platforms => [:jruby]
  gem 'jwt', '~> 1.5', '>= 1.5.6'
  gem 'multi_json', '~> 1.11.0'
  gem 'mime-types', '< 2.0.0'
  gem 'netrc', '~> 0.7.7'
  gem 'rb-fsevent', '~> 0.9'
  gem 'rspec', '~> 3.0.0'
  gem 'simplecov', :require => false
  gem 'vcr', '~> 4.0'
  gem 'webmock', '~> 3.4', '>= 3.4.2'
end

platforms :rbx do
  gem 'psych'
  gem 'rubysl', '~> 2.0'
end

gemspec

1
